In [1]:
#Run cell to mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# install package to have access to custom functions
%pip install /content/drive/MyDrive/EMG_gestures/ --use-feature=in-tree-build

Processing ./drive/MyDrive/EMG_gestures
  Created wheel for EMG-gestures: filename=EMG_gestures-0.1.0-py3-none-any.whl size=36722 sha256=a0f5cce1544cb42d86a2b0c940a3fd5470a6c6dc7e7d5f28a3a9731b823523d2
  Stored in directory: /tmp/pip-ephem-wheel-cache-lnrxrq9n/wheels/a2/b7/61/2147fa082a9e51bef5dcc38dd3f0898fe0554d62203c0e383e
Successfully built EMG-gestures
  Attempting uninstall: EMG-gestures
    Found existing installation: EMG-gestures 0.1.0
    Uninstalling EMG-gestures-0.1.0:
      Successfully uninstalled EMG-gestures-0.1.0


In [3]:
#import necessary packages

#our workhorses
import numpy as np
import pandas as pd
import scipy

#to visualize
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
#style params for figures
sns.set(font_scale = 2)
plt.style.use('seaborn-white')
plt.rc("axes", labelweight="bold")
from IPython.display import display, HTML

#to load files
import os
import sys
import h5py
import pickle
from tensorflow import keras

#append repo folder to search path
#import cusotm functions
from EMG_gestures.utils import *
from EMG_gestures.analysis import log_reg_xsubject_joint_data_train_frac_subjects, log_reg_xsubject_joint_data_train_all_subjects



In [4]:
#define hyper params for each model
model_dict = {1:{'n_dense_pre':0, 'activation':''},\
              2:{'n_dense_pre':1, 'activation':'tanh'},\
              3:{'n_dense_pre':1, 'activation':'relu'},\
              4:{'n_dense_pre':2, 'activation':'tanh'},\
              5:{'n_dense_pre':2, 'activation':'relu'},\
              }


In [5]:
#define where the data files are located
data_folder = '/content/drive/MyDrive/EMG_gestures/EMG_data/'

nsubjects = 36

#randomly-selected subjects to use as hold-out test data 
test_subjects = [17, 23,  7,  8,  3]

# User-defined parameters
lo_freq = 20 #lower bound of bandpass filter
hi_freq = 450 #upper bound of bandpass filter

win_size = 100 #define window size over which to compute time-domain features
step = win_size #keeping this parameter in case we want to re-run later with some overlap


In [6]:
#intialize empty lists
feature_matrix_all = np.empty((0,0))
target_labels_all = np.empty((0,))
window_tstamps_all = np.empty((0,))
block_labels_all  = np.empty((0,))
subject_id_all = np.empty((0,))
block_count = 0

for subject_id in range(1,nsubjects+1):
    if subject_id not in test_subjects:
        subject_folder = os.path.join(data_folder,'%02d'%(subject_id))
        print('=======================')
        print(subject_folder)

        # Process data and get features 
        #get features across segments and corresponding info
        feature_matrix, target_labels, window_tstamps, \
        block_labels, series_labels = get_subject_data_for_classification(subject_folder, lo_freq, hi_freq, \
                                                                        win_size, step)

        #prevent repeat of block labels by increasing block count
        block_labels = block_labels+block_count
        block_count = np.max([block_count, np.max(block_labels)])

        # concatenate lists
        feature_matrix_all = np.vstack((feature_matrix_all,feature_matrix)) if feature_matrix_all.size else feature_matrix
        target_labels_all = np.hstack((target_labels_all,target_labels))
        window_tstamps_all = np.hstack((window_tstamps_all,window_tstamps))
        block_labels_all = np.hstack((block_labels_all,block_labels))
        subject_id_all = np.hstack((subject_id_all,np.ones((block_labels.size))*subject_id))
        

/content/drive/MyDrive/EMG_gestures/EMG_data/01
/content/drive/MyDrive/EMG_gestures/EMG_data/02
/content/drive/MyDrive/EMG_gestures/EMG_data/04
/content/drive/MyDrive/EMG_gestures/EMG_data/05
/content/drive/MyDrive/EMG_gestures/EMG_data/06
/content/drive/MyDrive/EMG_gestures/EMG_data/09
/content/drive/MyDrive/EMG_gestures/EMG_data/10
/content/drive/MyDrive/EMG_gestures/EMG_data/11
/content/drive/MyDrive/EMG_gestures/EMG_data/12
/content/drive/MyDrive/EMG_gestures/EMG_data/13
/content/drive/MyDrive/EMG_gestures/EMG_data/14
/content/drive/MyDrive/EMG_gestures/EMG_data/15
/content/drive/MyDrive/EMG_gestures/EMG_data/16
/content/drive/MyDrive/EMG_gestures/EMG_data/18
/content/drive/MyDrive/EMG_gestures/EMG_data/19
/content/drive/MyDrive/EMG_gestures/EMG_data/20
/content/drive/MyDrive/EMG_gestures/EMG_data/21
/content/drive/MyDrive/EMG_gestures/EMG_data/22
/content/drive/MyDrive/EMG_gestures/EMG_data/24
/content/drive/MyDrive/EMG_gestures/EMG_data/25
/content/drive/MyDrive/EMG_gestures/EMG_

In [8]:
results_folder = '/content/drive/MyDrive/EMG_gestures/results_data/xsubject_joint_data/log_reg/'
figure_folder = '/content/drive/MyDrive/EMG_gestures/figures/training_history/xsubject_joint_data/log_reg'


#RNN training args - all other arguments are the same
verbose = 0
epochs = 30
batch_size = 5
validation_split = 0.1
# experiment params
n_splits = 4
nreps = 10

#excluded labels
exclude = [0,7]
#performance metrics
score_list = ['f1','accuracy']


for model_id in range(1,5+1):
    np.random.seed(1)# Set seed for replicability
    results_df = []
    for rep in range(nreps):
        print('Model %d | Rep %d'%(model_id, rep+1))
        print('--True Data--')
        rep_results_df, train_history = log_reg_xsubject_joint_data_train_frac_subjects(feature_matrix_all, target_labels_all, subject_id_all,\
                                                                                        block_labels_all, exclude,\
                                                                                        model_dict = model_dict[model_id], score_list = score_list,\
                                                                                        n_splits = n_splits,\
                                                                                        verbose = verbose, epochs = epochs, batch_size = batch_size,\
                                                                                        validation_split = validation_split, mv = None, permute = False)
        #add details and concatenate dataframe
        rep_results_df['Shuffled'] = False
        rep_results_df['Rep'] =  rep+1
        results_df.append(rep_results_df)


        #plot training history
        fig_title = 'Log reg model %02d; rep %i'%(model_id,rep)
        fig_fn = os.path.join(figure_folder,'log_reg_model_%02d_rep_%i_loss.png'%(model_id,rep))
        plot_training_history(train_history, fig_title,fig_fn)

        #repeat with shuffled data
        print('Model %d | Rep %d'%(model_id, rep+1))
        print('--Permuted Data--')
        rep_results_df, train_history = log_reg_xsubject_joint_data_train_frac_subjects(feature_matrix_all, target_labels_all, subject_id_all,\
                                                                                        block_labels_all, exclude,\
                                                                                        model_dict = model_dict[model_id], score_list = score_list,\
                                                                                        n_splits = n_splits,\
                                                                                        verbose = verbose, epochs = epochs, batch_size = batch_size,\
                                                                                        validation_split = validation_split, mv = None, permute = True)
        #add details and concatenate dataframe
        rep_results_df['Shuffled'] = True
        rep_results_df['Rep'] =  rep+1
        results_df.append(rep_results_df)

    #concatenate all data frames
    results_df = pd.concat(results_df,axis = 0)

    # #save results to file
    results_fn = 'model_%02d_results.h5'%(model_id)
    results_df.to_hdf(os.path.join(results_folder,results_fn), key='results_df', mode='w')


Model 1 | Rep 1
--True Data--
Split Count: 1
Training Model
Evaluate Model on Trained Data
Evaluate Model
Split Count: 2
Training Model
Evaluate Model on Trained Data
Evaluate Model
Split Count: 3
Training Model
Evaluate Model on Trained Data
Evaluate Model
Split Count: 4
Training Model
Evaluate Model on Trained Data
Evaluate Model
Model 1 | Rep 1
--Permuted Data--
Split Count: 1
Training Model
Evaluate Model on Trained Data
Evaluate Model
Split Count: 2
Training Model
Evaluate Model on Trained Data
Evaluate Model
Split Count: 3
Training Model
Evaluate Model on Trained Data
Evaluate Model
Split Count: 4
Training Model
Evaluate Model on Trained Data
Evaluate Model
Model 1 | Rep 2
--True Data--
Split Count: 1
Training Model
Evaluate Model on Trained Data
Evaluate Model
Split Count: 2
Training Model
Evaluate Model on Trained Data
Evaluate Model
Split Count: 3
Training Model
Evaluate Model on Trained Data
Evaluate Model
Split Count: 4
Training Model
Evaluate Model on Trained Data
Evaluate 

In [51]:
results_folder = '/content/drive/MyDrive/EMG_gestures/results_data/xsubject_joint_data/log_reg/'
model_dir = '/content/drive/MyDrive/EMG_gestures/model_data/xsubject_joint_data/log_reg/'
figure_dir = '/content/drive/MyDrive/EMG_gestures/figures/training_history/xsubject_joint_data/log_reg/'

#network training args 
verbose = 0
epochs = 30
batch_size = 2
validation_split = 0.1
nreps = 2

exclude = [0,7]

score_list = ['f1','accuracy']#performance metrics

model_id = 1
results_df = []
np.random.seed(1)
for rep in range(nreps):

    
    figure_folder = os.path.join(figure_dir,'rep_%i'%(rep+1))
    if not os.path.isdir(figure_folder):
        os.makedirs(figure_folder)
    model_folder = os.path.join(model_dir,'rep_%i'%(rep+1))
    if not os.path.isdir(model_folder):
        os.makedirs(model_folder)
    
    print('Model %i || Rep %02d'%(model_id, rep+1))
    print('----True Data----')
    rep_results_df, train_history, trained_model, scaler = log_reg_xsubject_joint_data_train_all_subjects(feature_matrix_all, target_labels_all,\
                                                                                                          subject_id_all,\
                                                                                                          block_labels_all, exclude,\
                                                                                                          model_dict = model_dict[model_id], score_list = score_list,\
                                                                                                          verbose = verbose, epochs = epochs, batch_size = batch_size,\
                                                                                                          validation_split = validation_split, mv = None, permute = False)
    #add details and concatenate dataframe
    rep_results_df['Shuffled'] = False
    rep_results_df['Rep'] =  rep+1
    results_df.append(rep_results_df)


    #plot training history
    fig_title = 'Log reg model %02d; rep %i'%(model_id,rep)
    fig_fn = os.path.join(figure_folder,'log_reg_model_%02d_rep_%i_loss.png'%(model_id,rep))
    plot_train_loss(train_history, fig_title,fig_fn)

    #save trained model
    model_fn = os.path.join(model_folder, 'trained_model_rep_%i_all_train_data.h5'%(rep))
    keras.models.save_model(trained_model, model_fn, save_format= 'h5')

    print('Model %i || Rep %02d'%(model_id, rep+1))
    print('----Permuted Data----')
    rep_results_df, a, b, c = log_reg_xsubject_joint_data_train_all_subjects(feature_matrix_all, target_labels_all,\
                                                                                                          subject_id_all,\
                                                                                                          block_labels_all, exclude,\
                                                                                                          model_dict = model_dict[model_id], score_list = score_list,\
                                                                                                          verbose = verbose, epochs = epochs, batch_size = batch_size,\
                                                                                                          validation_split = validation_split, mv = None, permute = True)
    #add details and concatenate dataframe
    rep_results_df['Shuffled'] = True
    rep_results_df['Rep'] =  rep+1
    results_df.append(rep_results_df)

results_df = pd.concat(results_df, axis = 0).reset_index()

#save scaler (outisde of rep loop b/c it will always be the same)
scaler_fn = 'trained_scaler_all_training_data.pkl'#
with open(os.path.join(model_dir,scaler_fn), "wb") as output_file:
    pickle.dump(scaler, output_file)

Model 1 || Rep 01
----True Data----
Training Model
Evaluate Model on Trained Data
Model 1 || Rep 01
----Permuted Data----
Training Model
Evaluate Model on Trained Data
Model 1 || Rep 02
----True Data----
Training Model
Evaluate Model on Trained Data
Model 1 || Rep 02
----Permuted Data----
Training Model
Evaluate Model on Trained Data
